In [1]:
import pandas as pd
import numpy as np

In [2]:
# Sample df_a: Indexed by time with some data in column 'B'
df_a = pd.DataFrame({
    'B': [100, 200, 300]
}, index=pd.to_datetime([
    '2025-04-10 08:30:00',
    '2025-04-10 09:00:00',
    '2025-04-10 09:30:00'
]))

# Sample df_b: 'time' column to match against df_a's index
df_b = pd.DataFrame({
    'time': [
        '2025-04-10 08:45:00',
        '2025-04-10 09:29:00',
        '2025-04-10 07:00:00'
    ]
})
df_b['time'] = pd.to_datetime(df_b['time'])

In [3]:
# Convert A's index to numpy array for efficient lookup
a_times = df_a.index.values

# Searchsorted gives index where each time would go in sorted a_times
closest_indices = np.searchsorted(a_times, df_b['time'].values, side='left')

# Helper function to pick the true closest between before/after
def get_best_idx(i, target):
    if i == 0:
        return i
    if i == len(a_times):
        return i - 1
    before = a_times[i - 1]
    after = a_times[i]
    return i - 1 if abs(target - before) < abs(target - after) else i

# Apply helper to get final matching indices
mapped_idx = [get_best_idx(i, t) for i, t in zip(closest_indices, df_b['time'].values)]

# Assign corresponding 'B' values to df_b
df_b['closest_B'] = df_a['B'].iloc[mapped_idx].values

print(df_b)

                 time  closest_B
0 2025-04-10 08:45:00        200
1 2025-04-10 09:29:00        300
2 2025-04-10 07:00:00        100
